In [76]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [48]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline

In [8]:
%run query_features.py
%run scripts/helper.py
%run scripts/model_train_plus_test.py

In [2]:
crowd_train = load_file('./data/train.csv/train.csv', None)
crowd_test = load_file('./data/test.csv/test.csv', None)

In [3]:
target = crowd_train.median_relevance.values

In [130]:
# train_index, test_index = ssSplit(target, train_size=8000, random_state=44)

In [147]:
# Xt = crowd_train.iloc[train_index]
# Xv = crowd_train.iloc[test_index]

Xt = crowd_train
Xv = crowd_test

In [148]:
# yt = target[train_index]
# yv = target[test_index]

yt = target

In [149]:
correct_map = build_query_correction_map(Xt, crowd_test)

In [150]:
def spell_correct_query(x):
    if x not in correct_map:
        return x
    else:
        return correct_map[x]
    
Xt['query'] = Xt['query'].map(spell_correct_query)

In [151]:
Xv['query'] = Xv['query'].map(spell_correct_query)

In [152]:
Xt_tweaked = tweak_text(Xt)
Xv_tweaked = tweak_text(Xv)

In [153]:
Xfitted, tfv = TFIDF(Xt_tweaked, None)

In [154]:
svd = TruncatedSVD(n_components=400, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)

scl = StandardScaler(copy=True, with_mean=True, with_std=True)

clf = SVC(C=10.0, kernel='rbf', degree=3, 
        gamma=0.0, coef0=0.0, shrinking=True, probability=False, 
        tol=0.001, cache_size=200, class_weight=None, 
        verbose=False, max_iter=-1, random_state=None)

keywords = keyword_counter(Xt)

In [155]:
features = stack([keywords, Xfitted])

In [156]:
pipeline = Pipeline([('svd', svd), ('scl', scl), ('clf', clf)])

In [157]:
pipeline.fit(features, yt)

MemoryError: 

In [ ]:
keywords_test = keyword_counter(Xv)

In [ ]:
Xtest = tfv.transform(Xv_tweaked)

In [ ]:
features_test = stack([keywords_test, Xtest])

In [ ]:
preds_new_model = pipeline.predict(features_test)

In [ ]:
make_submission(crowd_test.id.values.astype(int), preds_new_model, 'spell_correct_preds.csv')

In [146]:
print 'Kappa score on validation set ', (quadratic_weighted_kappa(yv, preds_new_model))

## Best score

In [126]:
best_score_df = pd.read_csv('./submissions/addtitional_features_500.csv')

In [127]:
best_score = best_score_df.prediction

In [128]:
ensemble = (preds_new_model + best_score) / 2
ensemble_int = [int(score) for score in ensemble]

In [129]:
make_submission(crowd_test.id.values.astype(int), ensemble_int, 'spell_correct.csv')